<a href="https://colab.research.google.com/github/Tanguyvans/HackAI/blob/main/comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch-pruning torchprofile

In [2]:
import sys
import torch.nn.functional as F
import torch
import torch_pruning as tp
import torchprofile
import os
import copy
import random
import numpy as np
import torch
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.transforms import *
from tqdm.auto import tqdm
from functools import partial
import torchvision.models as models
#assert torch.cuda.is_available()

In [3]:
# importing shared folder and existing code from HackIA24_input
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
folder='/content/drive/MyDrive/Workshop/' #@param ['/content/drive/MyDrive/Workshop/','/content/drive/MyDrive/IA/UMONS/Workshop/']

image_classification_path = f'{folder}2. Image classification'

train_dataset_path=   image_classification_path + '/' + 'data'
test_dataset_path = image_classification_path + '/' + 'test'

best_model_path = image_classification_path + '/' + "squeeze_final_25.pth"

Mounted at /content/drive


In [9]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.load( best_model_path ).cuda()

random_seed = 42

# pruning parameters
method = "random" # choices "random", "l1", "lamp", "slim", "group_norm", "group_sl"
speed_up = 2 # speed up ratio based on MACs
compression_ratio = 2 # compression ratio based on model size
global_pruning = True # if True: global pruning else: local
iterative_steps = 400 # pruning steps
max_sparsity = 1 # max sparsity

num_classes = 3 # to avoid pruning last layer

# Fixer le seed pour la reproductibilité
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# training parameters for iterative pruning
batchSize	 = 32                                                             # define the batch size
trainSplit   = 0.8                                                            # define the ratio of data to use for training
testSplit    = 0.1                                                            # define the ratio of data to use for testing
validSplit   = 0.1                                                            # the rest is used for validation
epochs		 = 100                                                            # define the number of epoch
criterion    = nn.CrossEntropyLoss()                                          # define the loss function
learnRate	 = 0.01                                                           # define the learning rate
optimizer    = Adam(model.parameters(), lr=learnRate)                # define the optimizer
imgSize      = 224

In [12]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
transform = transforms.Compose([
    transforms.RandomResizedCrop(imgSize),
    transforms.ToTensor()
])

target_transform = transforms.Compose([
    lambda label: torch.nn.functional.one_hot(torch.tensor(label), num_classes=3).float()
])

# Dataset
dataset = datasets.ImageFolder(train_dataset_path, transform=transform, target_transform=target_transform)

# Splitting the dataset
trainlen = int(trainSplit * len(dataset))
testlen = int(testSplit * len(dataset))
validlen = len(dataset) - trainlen - testlen

trainset, validset, testset = torch.utils.data.random_split(dataset, [trainlen, validlen, testlen])

# DataLoaders
trainloader = DataLoader(dataset, batch_size=batchSize, sampler=SubsetRandomSampler(trainset.indices))
valloader = DataLoader(dataset, batch_size=batchSize, sampler=SubsetRandomSampler(validset.indices))
testloader = DataLoader(dataset, batch_size=batchSize, sampler=SubsetRandomSampler(testset.indices))

# Example training loop
for inputs, labels in trainloader:
    inputs, labels = inputs.to(device), labels.to(device)
    # Perform your training operations here

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [13]:

# Evaluation loop
@torch.no_grad()
def evaluate(
    model: nn.Module,
    dataloader: DataLoader,
    verbose=True,
) -> float:
    model.eval()

    num_samples = 0
    num_correct = 0
    loss = 0

    for inputs, targets in tqdm(dataloader, desc="eval", leave=False, disable=not verbose):
        # Move the data from CPU to GPU
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Inference
        outputs = model(inputs)
        # Calculate loss
        loss += F.cross_entropy(outputs, targets, reduction="sum")
        # Convert logits to class indices
        outputs = outputs.argmax(dim=1)
        # Update metrics
        num_samples += targets.size(0)
        num_correct += (outputs == targets).sum()
    return (num_correct / num_samples * 100).item(), (loss / num_samples).item()

# training loop
def train(
    model: nn.Module,
    train_loader: DataLoader,
    test_loader: DataLoader,
    epochs: int,
    lr: int,
    # for pruning
    weight_decay=5e-4,
    pruner=None,
    callbacks=None,
    save=None,
    save_only_state_dict=False,
) -> None:

    optimizer = torch.optim.SGD(model.parameters(
    ), lr=lr, momentum=0.9, weight_decay=weight_decay if pruner is None else 0)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40,80,100], gamma=0.1)
    criterion = nn.CrossEntropyLoss()
    best_acc = -1
    best_checkpoint = dict()


    for epoch in range(epochs):
        model.train()
        for inputs, targets in tqdm(train_loader, leave=False):
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Reset the gradients (from the last iteration)
            optimizer.zero_grad()

            # Forward inference
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward propagation
            loss.backward()

            # Pruner regularize for sparsity learning
            if pruner is not None:
                pruner.regularize(model)

            # Update optimizer
            optimizer.step()

            if callbacks is not None:
                for callback in callbacks:
                    callback()

        acc, val_loss = evaluate(model, test_loader)
        print(
            f'Epoch {epoch + 1}/{epochs} | Val acc: {acc:.2f} | Val loss: {val_loss:.4f} | LR: {optimizer.param_groups[0]["lr"]:.6f}')

        if best_acc < acc:
            best_checkpoint['state_dict'] = copy.deepcopy(model.state_dict())
            best_acc = acc
        # Update LR scheduler
        scheduler.step()
    model.load_state_dict(best_checkpoint['state_dict'])
    if save:
        # on veut sauvegarder le meilleur modèle
        path = os.path.join(os.getcwd(), "results", save)
        os.makedirs(os.path.dirname(path), exist_ok=True)
        if save_only_state_dict:
            torch.save(model.state_dict(), path)
        else:
            torch.save(model, path)
    print(f'Best val acc: {best_acc:.2f}')



# Pruner
# définir le nbre de classses => évite de pruner la dernière couche
def get_pruner(model, example_input):
    sparsity_learning = False
    if method == "random":
        imp = tp.importance.RandomImportance()
        pruner_entry = partial(tp.pruner.MagnitudePruner, global_pruning=global_pruning)
    elif method == "l1":
        imp = tp.importance.MagnitudeImportance(p=1)
        pruner_entry = partial(tp.pruner.MagnitudePruner, global_pruning=global_pruning)
    elif method == "lamp":
        imp = tp.importance.LAMPImportance(p=2)
        pruner_entry = partial(tp.pruner.BNScalePruner, global_pruning=global_pruning)
    elif method == "slim":
        sparsity_learning = True
        imp = tp.importance.BNScaleImportance()
        pruner_entry = partial(tp.pruner.BNScalePruner, reg=1e-5, global_pruning=global_pruning)
    elif method == "group_norm":
        imp = tp.importance.GroupNormImportance(p=2)
        pruner_entry = partial(tp.pruner.GroupNormPruner, global_pruning=global_pruning)
    elif method == "group_sl":
        sparsity_learning = True
        imp = tp.importance.GroupNormImportance(p=2)
        pruner_entry = partial(tp.pruner.GroupNormPruner, reg=1e-5, global_pruning=global_pruning)
    else:
        raise NotImplementedError

    unwrapped_parameters = []
    ignored_layers = []
    ch_sparsity_dict = {}
    # ignore output layers
    for m in model.modules():
        if isinstance(m, torch.nn.Linear) and m.out_features == num_classes:
            ignored_layers.append(m)
        elif isinstance(m, torch.nn.modules.conv._ConvNd) and m.out_channels == num_classes:
            ignored_layers.append(m)


    # Here we fix iterative_steps=200 to prune the model progressively with small steps
    # until the required speed up is achieved.
    pruner = pruner_entry(
        model,
        example_input,
        importance=imp,
        iterative_steps=iterative_steps,
        ch_sparsity=1,
        ch_sparsity_dict=ch_sparsity_dict,
        ignored_layers=ignored_layers,
        unwrapped_parameters=unwrapped_parameters,
    )
    return pruner

# pruning jusqu'à atteindre le speed up voulu
def progressive_pruning_speedup(pruner, model, speed_up, example_inputs):
    model.eval()
    base_ops, _ = tp.utils.count_ops_and_params(
        model, example_inputs=example_inputs)
    current_speed_up = 1
    while current_speed_up < speed_up:
        pruner.step(interactive=False)
        pruned_ops, _ = tp.utils.count_ops_and_params(
            model, example_inputs=example_inputs)
        current_speed_up = float(base_ops) / pruned_ops
        # print(current_speed_up)
        if pruner.current_step == pruner.iterative_steps:
            break
    return current_speed_up


# pruning jusqu'à atteindre le ratio de compression voulu
def progressive_pruning_compression_ratio(pruner, model, compression_ratio, example_inputs):
    # compression ratio défini par taille initiale / taille finale
    model.eval()
    _, base_params = tp.utils.count_ops_and_params(
        model, example_inputs=example_inputs)
    current_compression_ratio = 1
    while current_compression_ratio < compression_ratio:
        pruner.step(interactive=False)
        _, pruned_params = tp.utils.count_ops_and_params(
            model, example_inputs=example_inputs)
        current_compression_ratio = float(base_params) / pruned_params
        if pruner.current_step == pruner.iterative_steps:
            break
        # print(current_compression_ratio)
    return current_compression_ratio



def main():
    # Avant pruning
    example_input = torch.rand(1, 3, 224, 224).to(device)
    start_macs, start_params = tp.utils.count_ops_and_params(model, example_input)
    print('----- Avant pruning -----')
    print(f'Nombre de MACs = {start_macs/1e6:.3f} M')
    print(f'Nombre de paramètres = {start_params/1e6:.3f} M')
    print('')

    pruner = get_pruner(model, example_input)
    print(f'Pruning method = {method}')

    # pruning using compression ratio as objective
    # progressive_pruning_compression_ratio(pruner, model, compression_ratio, example_input)

    # pruning using speed up ratio as objective (Number of MACs)
    progressive_pruning_speedup(pruner, model, speed_up, example_input)
    pruned_macs, pruned_params = tp.utils.count_ops_and_params(model, example_input)
    print('----- Après pruning -----')
    print(f'Nombre de MACs = {pruned_macs/1e6:.3f} M')
    print(f'Nombre de paramètres = {pruned_params/1e6:.3f} M')
    print('')

    # Results
    print('----- Results before fine tuning -----')
    print(f'Params: {start_params/1e6:.2f} M => {pruned_params/1e6:.2f} M')
    print(f'MACs: {start_macs/1e6:.2f} M => {pruned_macs/1e6:.2f} M')
    print('')

In [18]:
example_input = torch.rand(1, 3, 224, 224)
start_macs, start_params = tp.utils.count_ops_and_params(model.cuda(), example_input.cuda())

pruner = get_pruner(model, example_input.cuda())
print(f'Pruning method = {method}')

# pruning using compression ratio as objective
#progressive_pruning_compression_ratio(pruner, model, compression_ratio, example_input)

# pruning using speed up ratio as objective (Number of MACs)
progressive_pruning_speedup(pruner, model, speed_up, example_input.cuda())
pruned_macs, pruned_params = tp.utils.count_ops_and_params(model, example_input.cuda())

# Results
print('----- Results before fine tuning -----')
print(f'Params: {start_params/1e6:.2f} M => {pruned_params/1e6:.2f} M')
print(f'MACs: {start_macs/1e6:.2f} M => {pruned_macs/1e6:.2f} M')
print('')

pruned_model_path = models_path + best_model[ :-4 ] + "-pruned" + best_model[ -4: ]
print( pruned_model_path )
torch.save( model, pruned_model_path )

/usr/local/lib/python3.10/dist-packages/torch_pruning/pruner/algorithms/metapruner.py:87: UserWarning: ch_sparsity is deprecated in v1.3.0. Please use pruning_ratio.
  warnings.warn("ch_sparsity is deprecated in v1.3.0. Please use pruning_ratio.")
/usr/local/lib/python3.10/dist-packages/torch_pruning/pruner/algorithms/metapruner.py:90: UserWarning: ch_sparsity_dict is deprecated in v1.3.0. Please use pruning_ratio_dict instead.
  warnings.warn("ch_sparsity_dict is deprecated in v1.3.0. Please use pruning_ratio_dict instead.")


Pruning method = random
----- Results before fine tuning -----
Params: 0.72 M => 0.72 M
MACs: 269.09 M => 269.09 M



NameError: name 'best_model' is not defined

In [ ]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train( model.cuda(), trainloader, testloader, epochs, learnRate, pruner = pruner, save = True)

In [ ]:
import sys


input_data = torch.rand(1, 3, 224, 224).to(device)
benchmark( model, input_data )